In [1]:
import torch
%run /workspaces/jupyterlite/content/pytroch/arrays_and_matricies.py

### Algorithms

#### One-Dimensional Array

In [2]:
def fill_zero_places(a):
    """
    a = tensor([1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
    b = tensor([-1,  1,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0, 0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0])
    nonzero_indices = tensor([ 0,  1,  6,  8, 14, 23])
    diff = tensor([1, 5, 2, 6, 9, 8])
    c = tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
    d = tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
    """
    b = torch.cat([torch.tensor([-1], device=a.device), a])
    nonzero_indices = torch.nonzero(b).squeeze()
    diff = torch.diff(torch.cat([nonzero_indices, torch.tensor([b.shape[0]], device=b.device)]))
    c = b[nonzero_indices].repeat_interleave(diff)
    c[c < 0] = 0
    d = c[1:]
    
    return d

In [3]:
def shift_values_to_next_available_zero(a):
    """
    a = tensor([1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
    b = tensor([1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
    c = tensor([ 0,  1,  0,  0,  0,  0,  6,  0,  8,  0,  0,  0,  0,  0, 14,  0,  0,  0, 0,  0,  0,  0,  0, 23,  0,  0,  0,  0,  0,  0,  0])
    d = tensor([ 1,  0,  0,  0,  0,  6,  0,  8,  0,  0,  0,  0,  0, 14,  0,  0,  0,  0, 0,  0,  0,  0, 23,  0,  0,  0,  0,  0,  0,  0])
    e = tensor([ 0,  1,  1,  1,  1,  1,  6,  6,  8,  8,  8,  8,  8,  8, 14, 14, 14, 14, 14, 14, 14, 14, 14, 23, 23, 23, 23, 23, 23, 23])
    f = tensor([1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    """
    b = (a > 0).int()
    b = torch.cat([torch.tensor([1], device=a.device), b])
    c = torch.arange(b.shape[0], device=a.device) * b
    d = c[1:]
    e = torch.roll(fill_zero_places(d), 1)
    e[0] = 0
    f = torch.zeros_like(a)
    f.scatter_add_(0, e.long(), a)
        
    return f

In [4]:
def fill_zero_values_ascending_rest_with_zero(a):
    """
    a = tensor([4, 3, 2, 1, 0, 1, 0, 5, 4, 3, 2, 1, 0, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 0, 4, 3, 2, 1, 0, 0])
    b = tensor([1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0], dtype=torch.int32)
    c = tensor([0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, 5, 5, 5, 5, 6, 7])
    d = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 2])
    """
    b = (a > 0).int()
    c = torch.cumsum(1 - b, dim=0)
    d = (c * (1 - b) - torch.maximum(b * c, torch.zeros_like(c)).cummax(dim=0)[0]) * (1 - b)
    
    return d

In [5]:
def find_one_zero_pattern(a):
    """
    a = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
    b = tensor([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1], dtype=torch.int32)
    c = tensor([ 0,  0,  0, -1,  1, -1,  1,  0,  0,  0,  0, -1,  1,  0,  0,  0,  0,  0, 0,  0, -1,  1, -1,  1,  0,  0,  0,  0,  0, -1], dtype=torch.int32)
    d = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
    """
    b = torch.roll(a.int(), -1)
    b[-1] = 1
    c = a - b
    d = (c == 1).int()
    
    return d

In [6]:
def fill_zero_values_descending_rest_with_zero(a):
    """
    a = tensor([4, 3, 2, 1, 0, 1, 0, 5, 4, 3, 2, 1, 0, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 0, 4, 3, 2, 1, 0, 0])
    b = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 2])
    c = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1], dtype=torch.int32)
    d = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
    e = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0])
    f = tensor([1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
    g = tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
    h = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0])
    """
    b = fill_zero_values_ascending_rest_with_zero(a)
    c = (b > 0).int()
    d = find_one_zero_pattern(c)
    e = d * b
    f = shift_values_to_next_available_zero(e)
    g = fill_zero_places(f)
    h = ((g + 1) - b) * c
    
    return h

In [7]:
def find_values_that_must_be_reordered(a):
    """
    a = tensor([-82, -83, -88, -92, -86, -91, -89, -94, -89, -93, -93, -95, -96, -100, -96, -104, -103, -98, -100, -100, -100, -109, -104, -110, -105, -114, -89, -85, -106, -58])
    b = tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])
    c = tensor([-83, -85, -91, -96, -91, -97, -96, -102, -98, -103, -104, -107, -109, -114, -111, -120, -120, -116, -119, -120, -121, -131, -127, -134, -130, -140, -116, -113, -135, -88])
    d = tensor([-140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -140, -135, -135, -135, -88])
    e = tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1], dtype=torch.int32)
    """
    b = torch.arange(1, a.shape[0] + 1)
    c = a - b
    d, _ = torch.cummin(c.flip(0), dim=0)
    d = d.flip(0)
    e = (c == d).int()

    return 1 - e

In [8]:
def adjust_cars_no_lane_change(a):
    """
    a = tensor([ 3, 10,  8, 11,  5, 13,  7, 14, 18, 15, 19, 20, 13, 21, 16, 24, 19, 24, 23, 22, 28, 22, 27, 26, 33, 32, 30, 32, 31, 36])
    b = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1], dtype=torch.int32)
    c = tensor([4, 3, 2, 1, 0, 1, 0, 5, 4, 3, 2, 1, 0, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 0, 4, 3, 2, 1, 0, 0])
    d = tensor([ 0,  0,  0,  0,  5,  0,  7,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0, 0,  0,  0, 22,  0, 26,  0,  0,  0,  0, 31, 36])
    e = tensor([ 5,  0,  0,  0,  0,  7,  0, 13,  0,  0,  0,  0,  0, 22,  0,  0,  0,  0, 0,  0,  0,  0, 26,  0, 31,  0,  0,  0,  0, 36])
    f = tensor([ 5,  5,  5,  5,  5,  7,  7, 13, 13, 13, 13, 13, 13, 22, 22, 22, 22, 22, 22, 22, 22, 22, 26, 26, 31, 31, 31, 31, 31, 36])
    g = tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 25, 26, 27, 28, 29, 30, 31, 36])
    """
    b = 1 - find_values_that_must_be_reordered(a)
    c = fill_zero_values_descending_rest_with_zero(b)
    d = a * b
    e = shift_values_to_next_available_zero(d)
    f = fill_zero_places(e)
    g = f - c

    return g

#### Matrices

In [9]:
a = torch.tensor([ 3, 10,  8, 11,  5, 13,  7, 14, 18, 15, 19, 20, 13, 21, 16, 24, 19, 24, 23, 22, 28, 22, 27, 26, 33, 32, 30, 32, 31, 36])
b = 1 - find_values_that_must_be_reordered(a)
c = fill_zero_values_descending_rest_with_zero(b)
d = a * b
e = shift_values_to_next_available_zero(d)
f = fill_zero_places(e)
g = f - c

a, b, c, d, e, f, g

(tensor([ 3, 10,  8, 11,  5, 13,  7, 14, 18, 15, 19, 20, 13, 21, 16, 24, 19, 24,
         23, 22, 28, 22, 27, 26, 33, 32, 30, 32, 31, 36]),
 tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
         0, 0, 0, 0, 1, 1], dtype=torch.int32),
 tensor([4, 3, 2, 1, 0, 1, 0, 5, 4, 3, 2, 1, 0, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 0,
         4, 3, 2, 1, 0, 0]),
 tensor([ 0,  0,  0,  0,  5,  0,  7,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,
          0,  0,  0, 22,  0, 26,  0,  0,  0,  0, 31, 36]),
 tensor([ 5,  0,  0,  0,  0,  7,  0, 13,  0,  0,  0,  0,  0, 22,  0,  0,  0,  0,
          0,  0,  0,  0, 26,  0, 31,  0,  0,  0,  0, 36]),
 tensor([ 5,  5,  5,  5,  5,  7,  7, 13, 13, 13, 13, 13, 13, 22, 22, 22, 22, 22,
         22, 22, 22, 22, 26, 26, 31, 31, 31, 31, 31, 36]),
 tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19, 20, 21, 22, 25, 26, 27, 28, 29, 30, 31, 36]))

### Use-Cases

#### Single Lane

In [10]:
a = torch.tensor([ 83,  84,  89,  93,  87,  92,  90,  95,  90,  94,  94,  96,  97, 101,  97, 105, 104,  99, 101, 101, 101, 110, 105, 111, 106, 115, 90,  86, 107,  59])
adjust_cars_no_lane_change(a)

tensor([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
        48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59])

In [11]:
a = torch.tensor([ 3, 10,  8, 11,  5, 13,  7, 14, 18, 15, 19, 20, 13, 21, 16, 24, 19, 24, 23, 22, 28, 22, 27, 26, 33, 32, 30, 32, 31, 36])
adjust_cars_no_lane_change(a)

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 25, 26, 27, 28, 29, 30, 31, 36])

In [12]:
a = torch.tensor([ 1,  4,  3,  6,  5,  6,  8,  8, 14, 10, 11, 13, 13, 14, 15, 16, 17, 18,  19, 20, 21, 23, 23, 24, 25, 26, 27, 28, 30, 35])
adjust_cars_no_lane_change(a)

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 35])

#### Multi-Lane

In [13]:
# [lane, index, speed]
cars = torch.tensor([[0, 1, 4], [1, 5, 3], [0, 3, 5], [1, 7, 3], [1, 6, 7], [0, 4, 3], [1, 10, 1], [1, 11, 3], [0, 5, 5], [0, 6, 3]])

In [14]:
(cars[1:, 1] - cars[:-1, 1]  == 0).int()

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)

In [15]:
def eligible_for_takeover(a, val_idx=1):
    """
    two lane case, assuming `a` is sorted by the idx-values
    """

    b = (a[1:, val_idx] - a[:-1, val_idx]  == 0).int()
    c = torch.roll(b, 1) | b
    d = 1 - c
    
    return d

In [16]:
val_idx=1
a = torch.tensor([[0, 1, 4], [0, 5, 3], [0, 3, 5], [1, 7, 3], [1, 6, 7], [0, 4, 3], [1, 10, 1], [1, 11, 3], [0, 5, 5], [0, 6, 3]], dtype=torch.float)
b = (a[1:, val_idx] - a[:-1, val_idx]  == 0).int()
c = torch.roll(b, 1) | b
d = 1 - c

a, b, c, d

(tensor([[ 0.,  1.,  4.],
         [ 0.,  5.,  3.],
         [ 0.,  3.,  5.],
         [ 1.,  7.,  3.],
         [ 1.,  6.,  7.],
         [ 0.,  4.,  3.],
         [ 1., 10.,  1.],
         [ 1., 11.,  3.],
         [ 0.,  5.,  5.],
         [ 0.,  6.,  3.]]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32))

In [17]:
def cars_desiring_takeover():
    """
    """
    
    pass

In [18]:
def cars_after_takeovers():
    """
    """
    
    pass

In [19]:
import torch

cars = torch.tensor([[0, 1, 4], [0, 5, 3], [0, 3, 5], [1, 7, 3], [1, 6, 7], [0, 4, 3], [1, 10, 1], [1, 11, 3], [0, 5, 5], [0, 6, 3]], dtype=torch.float)
max_values = maximum_value_by_grp(cars, 0, 1)

cars, max_values

(tensor([[ 0.,  1.,  4.],
         [ 0.,  5.,  3.],
         [ 0.,  3.,  5.],
         [ 1.,  7.,  3.],
         [ 1.,  6.,  7.],
         [ 0.,  4.,  3.],
         [ 1., 10.,  1.],
         [ 1., 11.,  3.],
         [ 0.,  5.,  5.],
         [ 0.,  6.,  3.]]),
 tensor([[ 0.,  6.],
         [ 1., 11.]]))

In [20]:
add_to_matrix_mapping_values(cars, max_values)

tensor([[ 0.,  7.,  4.],
        [ 0., 11.,  3.],
        [ 0.,  9.,  5.],
        [ 1., 24.,  3.],
        [ 1., 23.,  7.],
        [ 0., 10.,  3.],
        [ 1., 27.,  1.],
        [ 1., 28.,  3.],
        [ 0., 11.,  5.],
        [ 0., 12.,  3.]])

In [21]:
cars_sort = torch.empty(cars.shape[0], 2)
cars_sort[:, 0] = cars[:, 0]
cars_sort[:, 1] = cars[:, 1]
cars_sort

tensor([[ 0.,  7.],
        [ 0., 11.],
        [ 0.,  9.],
        [ 1., 24.],
        [ 1., 23.],
        [ 0., 10.],
        [ 1., 27.],
        [ 1., 28.],
        [ 0., 11.],
        [ 0., 12.]])

In [22]:
cars = torch.tensor([[0, 1, 4], [0, 5, 3], [0, 3, 5], [1, 7, 3], [1, 6, 7], [0, 4, 3], [1, 10, 1], [1, 11, 3], [0, 5, 5], [0, 6, 3]], dtype=torch.float)
max_values = maximum_value_by_grp(cars, 0, 1)

a = add_to_matrix_mapping_values(cars, max_values)
b = group_ordering_based_on_sorting(a)
c = sort_matrix_by_nth_and_mth_column(a)
# b = adjust_cars_no_lane_change(a)
a, b, c

(tensor([[ 0.,  7.,  4.],
         [ 0., 11.,  3.],
         [ 0.,  9.,  5.],
         [ 1., 24.,  3.],
         [ 1., 23.,  7.],
         [ 0., 10.,  3.],
         [ 1., 27.,  1.],
         [ 1., 28.,  3.],
         [ 0., 11.,  5.],
         [ 0., 12.,  3.]]),
 tensor([0., 1., 2., 5., 8., 9., 3., 4., 6., 7.]),
 tensor([[ 0.,  7.,  4.],
         [ 0.,  9.,  5.],
         [ 0., 10.,  3.],
         [ 0., 11.,  3.],
         [ 0., 11.,  5.],
         [ 0., 12.,  3.],
         [ 1., 23.,  7.],
         [ 1., 24.,  3.],
         [ 1., 27.,  1.],
         [ 1., 28.,  3.]]))

In [23]:
c[b.int()]

tensor([[ 0.,  7.,  4.],
        [ 0.,  9.,  5.],
        [ 0., 10.,  3.],
        [ 0., 12.,  3.],
        [ 1., 27.,  1.],
        [ 1., 28.,  3.],
        [ 0., 11.,  3.],
        [ 0., 11.,  5.],
        [ 1., 23.,  7.],
        [ 1., 24.,  3.]])

In [24]:
import torch

# Create the matrix as a torch tensor
matrix = torch.tensor([[ 0.,  2.],
        [ 0., 11.],
        [ 0.,  3.],
        [ 1.,  4.],
        [ 1.,  0.],
        [ 1.,  3.]])

# Transpose the matrix
transposed = matrix.T

# Sort by first column, then second column
sorted_indices = torch.argsort(transposed[0] * matrix.shape[0] + transposed[1])
sorted_array = matrix[sorted_indices]

# The result is in sorted_array
print(sorted_array)

tensor([[ 0.,  2.],
        [ 0.,  3.],
        [ 1.,  0.],
        [ 1.,  3.],
        [ 1.,  4.],
        [ 0., 11.]])


### Benchmarks

In [25]:
count = 30000
lane_num_tracks = torch.tensor([300000])
cars_position = torch.arange(0, count)
cars_speed = torch.randint(0,10, (count, 1)).flatten()

adjust_cars_no_lane_change(cars_position + cars_speed)

tensor([    0,     1,     2,  ..., 29999, 30000, 30001])

In [26]:
# import torch
# import time

# if not torch.cuda.is_available():
#     raise RuntimeError("CUDA is not available, please run on a CUDA-capable device.")

# iterations = 100000
# count = 30000

# lane_num_tracks = torch.tensor([300000000]).cuda()
# cars_position = torch.arange(0, count).cuda()
# cars_speed = torch.randint(0, 10, (count, 1)).flatten().cuda()

# def adjust_cars_no_lane_change(cars_info):
#     # Sample implementation, replace with the actual function
#     return cars_info * 2  # Example operation

# # Start timing
# start_time = time.time()

# # Execute the function multiple times
# for _ in range(iterations):
#     result = adjust_cars_no_lane_change(cars_position + cars_speed)

# # Optionally, synchronize and free the cache to get accurate timings
# torch.cuda.synchronize()
# torch.cuda.empty_cache()

# # End timing
# end_time = time.time()

# # Calculate elapsed time
# elapsed_time = end_time - start_time
# print(f"Elapsed time: {elapsed_time} seconds")

Iterationen: 1.000.000
Autos: 300.000

Zeit: 16 Sekunden

Iterationen: 10.000.000
Autos: 30.000

Zeit: 94 Sekunden

Iterationen: 1.000.000
Autos: 3.000.000

Zeit: 178 Sekunden